# General imports and functions

In [1]:
#Imports - general
import random
random.seed(15)
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('paper')
from math import ceil, sqrt
#import sklearn
from sklearn.metrics import f1_score, roc_curve, auc, precision_score, accuracy_score
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
%matplotlib inline

#Imports - RDKit
from rdkit.Chem import MolFromSmiles
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.Chem.rdMolDescriptors import *

#Imports - additional
from os import listdir #for getting a list of files in a dir to process checkpoints
import time # for real-time timing the progress of the network

path = 'data/targets/'   #path to .ism files
batch_number = 0
#Get the list of filenames in data/targets/
#ordered by the size of class, descending:
with open("receptors_desc.txt", "r") as f:
    receptors = [l.strip().split() for l in f.readlines()]
#receptors[:5]

#---

# Functions

## Preparing the dataset

def read_ism(file_name):
    ''' Parse an .ism file, returning a list of smiles of molecules '''
    mol_list = []
    with open(file_name, 'r') as f:
        frl = f.readlines()
    for line in frl:
        line = line.split('\t')
        smile = line[0]
        mol_list.append(smile)
    return mol_list

#### Creating molecule representations

def get_class_vectors(mol_matrix, rType = 'fingerprint', f_size=2048):
    ''' For each molecule in the array (list of lists):
        create its vector representation for training (fingerprint/descriptor/mixed)
        and a binary vector showing which classes it belongs to.
        f_size - length of the fingerprint
    '''
    cv_dict = {}
    for i,l in enumerate(mol_matrix):
        for smile in l:
            if smile not in cv_dict:
                
                mol = MolFromSmiles(smile)
                if rType == 'fingerprint':
                    rep = fingerprint(mol, f_size)
                elif rType == 'descriptor':
                    rep = descriptor(mol)
                else: # both representations
                    rep = descriptor(mol) + list(fingerprint(mol,f_size))
                
                labels = [0]*len(mol_matrix)
                cv_dict[smile] = [rep, labels]
            cv_dict[smile][1][i] = 1
    # Normalize descriptors
    if rType != 'fingerprint':
        matrix_to_norm = []
        for vals in cv_dict.values():
            matrix_to_norm.append(vals[0][:37])
        matrix_to_norm = np.array(matrix_to_norm)
        v_min = matrix_to_norm.min(axis=0)
        v_max = matrix_to_norm.max(axis=0) - v_min
        
        for key in cv_dict.keys():
            for i in range(37):
                normed_val = (cv_dict[key][0][i] - v_min[i]) / v_max[i]
                cv_dict[key][0][i] = normed_val
        
    return cv_dict

def fingerprint(mol, f_size=2048):
    return GetMorganFingerprintAsBitVect(mol,2,f_size)

def descriptor(mol):
    functions = [CalcChi0n,
            CalcChi0v,
            CalcChi1n,
            CalcChi1v,
            CalcChi2n,
            CalcChi2v,
            CalcChi3n,
            CalcChi3v,
            CalcChi4n,
            CalcChi4v,
            CalcExactMolWt,
            CalcFractionCSP3,
            CalcHallKierAlpha,
            CalcKappa1,
            CalcKappa2,
            CalcKappa3,
            CalcLabuteASA,
            CalcNumAliphaticCarbocycles,
            CalcNumAliphaticHeterocycles,
            CalcNumAliphaticRings,
            CalcNumAmideBonds,
            CalcNumAromaticCarbocycles,
            CalcNumAromaticHeterocycles,
            CalcNumAromaticRings,
            CalcNumBridgeheadAtoms,
            CalcNumHBA,
            CalcNumHBD,
            CalcNumHeteroatoms,
            CalcNumHeterocycles,
            CalcNumLipinskiHBA,
            CalcNumLipinskiHBD,
            CalcNumRings,
            CalcNumSaturatedCarbocycles,
            CalcNumSaturatedHeterocycles,
            CalcNumSaturatedRings,
            CalcNumSpiroAtoms,
            CalcTPSA]
        
    descriptors = []
    for function in functions:
        descriptors.append(function(mol))
    return descriptors

def prepare_sets(mol_matrix, val_prc=0.25, rType='fingerprint', f_size=2048):
    ''' Create training and validation sets with labels from an array of smiles
        rType - type of representation - fingerprint/descriptor/mixed
        f_size - size of the fingerprint if rType is fingerprint or mixed '''
    # Create a representation and a class vector for each molecule
    molecules = get_class_vectors(mol_matrix,rType,f_size)

    # Split the data into training and validation sets
    mts=[[],[]]
    for val in molecules.values():
        mts[0].append(val[0]) # [fingerprint]
        mts[1].append(val[1]) # [label]

    molecules_split = train_test_split(mts[0], mts[1], test_size=val_prc, random_state=15)

    # Validation set:
    #     a list of representations (mols) and
    #     a list of corresponding class vectors (labels)
    val_mols = np.array(molecules_split[1])
    val_labels = np.array(molecules_split[3])
    # Training set:
    train_set = [[],[]]
    train_set[0] = np.array(molecules_split[0])
    train_set[1] = np.array(molecules_split[2])
    
    return train_set, val_mols, val_labels

## Neuron layers

def neuron_layer(isize, hsize, prev_layer):
    ''' Create a single neuron layer - weight, bias, placeholder '''
    # weights and biases
    w = tf.Variable(tf.random_normal((isize, hsize), stddev=1/sqrt(isize)))
    b = tf.Variable(tf.random_normal((hsize,), stddev=0.1))
    # neuron
    h = tf.nn.relu(tf.matmul(prev_layer, w) + b)
    return w,b,h

def setup_layers(layers, osize, isize=2048):
    ''' Create placeholders, weights and biases for all requested layers '''
    x = tf.placeholder(tf.float32, shape=[None,isize])
    active_layers = {'x':x,'w':[],'b':[],'h':[]}
    
    for i in range(len(layers)):
        if i == 0:
            w,b,h = neuron_layer(isize, layers[i], x)
        else:
            w,b,h = neuron_layer(layers[i-1], layers[i], prev_h)
        active_layers['w'].append(w)
        active_layers['b'].append(b)
        active_layers['h'].append(h)
        prev_h = h
    
    # Output Layer
    wo = tf.Variable(tf.random_normal((layers[-1], osize), stddev=1/sqrt(layers[-1])))
    bo = tf.Variable(tf.random_normal((osize,), stddev=0.1))
    a = tf.matmul(h, wo) + bo #h is the one last initialized in the loop > of the last layer
    
    # Placeholder for targets
    t = tf.placeholder(tf.float32, shape=[None, osize])
    
    active_layers['out'] = [wo,bo,a,t]
    return active_layers

## Training functions

def next_batch(data, size): #data = [[fingerprints], [labels]]
    ''' Extract the next batch from a dataset.
        If dataset size is not dividable by batch size with reminder 0,
       the last full batch and the remaining incomplete batch will be merged 
       to incorporate all data.'''
    global batch_number
    if batch_number == len(data[0])//size-1:
        #the last batch may be larger to incorporate all data
        # -1 because the first batch number is 0
        start = batch_number*size
        batch_number = 0
        return data[0][start:], data[1][start:]
    start = batch_number*size
    batch_number += 1
    return data[0][start:start+size], data[1][start:start+size]

def shuffle_data(data):
    '''Used to shuffle the training set after an epoch'''
    indices = list(range(len(data[0])))
    random.shuffle(indices)
    new_data = [data[0][indices], data[1][indices]]
    return new_data

def timer(start,end):
    ''' For timing the training of the network '''
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    return "{:0>2}:{:0>2}:{:0>2}".format(int(hours),int(minutes),int(seconds))

def tf_fractions(a1, a2):
    ''' Calculate true/false positives/negatives in a2 based on a1
        for each class seperately. '''
    num_class = len(a1[0]) #number of classes
    tp = np.array([0] * num_class)
    fn = np.array([0] * num_class)
    tn = np.array([0] * num_class)
    fp = np.array([0] * num_class)
    for i in range(num_class):
        for j in range(len(a1)):
            if a2[j][i] == a1[j][i]:
                if a1[j][i] == 1:
                    tp[i] += 1
                else:
                    tn[i] += 1
            else:
                if a1[j][i] == 1:
                    fn[i] += 1
                else:
                    fp[i] += 1
    return [tp,fn,tn,fp]

def train_network(data, batch_size, epochs, verbose=True, save=True):
    ''' This function trains the network, returning the rate of training and validation accuracy '''
    # Variables:
    global start, train, val_pred, batch_number
    iterations = len(data[0]) // batch_size * epochs
    if verbose: print(iterations, ' iterations')
    tr_rate = []
    val_rate = []
    f1 = [[],[],[]]
    tf_frac = []
    batch_number = 0
    save_step=0
    # Training:
    with tf.Session() as session:
        session.run(tf.initialize_all_variables())

        if save:
            saver.save(session, "tmp/checkpoint", global_step=save_step)
        
        # Train the network
        for i in range(iterations):
            if i != 0 and batch_number == 0:
                data = shuffle_data(data)
            
            mols, labels = next_batch(data, batch_size)
            
            session.run(train, feed_dict={x: mols, t: labels})
            
            # Display progress and make a checkpoint
            if i == 0 or (i+1)%(iterations//10) == 0:
                if save:
                    save_step += 1
                    saver.save(session, "tmp/checkpoint", global_step = save_step)
                if verbose:
                    progress = int((i+1)/(iterations//10))*10
                    now = timer(start,time.time())
                    print("%3i" % progress + '%, ' + str(now))                
            
            # Validate predictions
            if i % 200 == 0 or i == iterations-1:
                
                tr_target = labels
                tr_predictions = session.run(predict, feed_dict={x: mols, t: labels})
                tr_rate.append((tr_target == tr_predictions).all(axis=1).mean())

                val_predictions = session.run(predict, feed_dict={x: val_mols, t: val_labels})
                val_rate.append((val_labels == val_predictions).all(axis=1).mean())
                
                
                f1[0].append(f1_score(val_labels, val_predictions, average='micro'))
                f1[1].append(f1_score(val_labels, val_predictions, average='macro'))
                f1[2].append(f1_score(val_labels, val_predictions, average='weighted'))
                
        tf_frac = tf_fractions(val_labels,val_predictions)
        # Final checkpoint
        if save:
            saver.save(session, "tmp/checkpoint", global_step=save_step+1)
    return tr_rate, val_rate, f1, tf_frac

def filename_gen(plot_type='', path=''):
    ''' Generate filename for a plot. '''
    if plot_type != '':
        plot_type += '_'
    filename = plot_type
    layers_str = "{}'{}".format(len(layers),max(layers))
    filename += "{}_{}_{}_{:>6}_{}_{}_{:.4}_{}".format(
                num_class, f_size, layers_str, learning_rate, batch_size, epochs, max(val_rate), rType)
    filename = filename.replace(".","'")
    filename = filename.replace(" ","_")
    filename = path + filename + ".pdf"
    return filename

def save_accuracy_plot(tr_rate, save=True, path="plots/"):
    ''' Draws and saves a plot which displays changes of accuracy 
        during training for training and validation sets. '''
    plt.close()
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    x = [i*200 for i in range(len(tr_rate))]
    ax.plot(x,tr_rate, label='trening')
    ax.plot(x,val_rate, label='walidacja')
    ax.legend(loc=5)
    ax.set_ylabel('Dokładność')
    ax.set_xlabel('Krok')
    plt.ylim([0,1])
    vals = ax.get_yticks()
    ax.set_yticklabels(['{:3.0f}%'.format(x*100) for x in vals])
    
    title = "Cele terapeutyczne: {:2}, Szybkość uczenia: {}\nWarstwy: {}, Rozmiar batcha: {}, Liczba epok: {}".format(
        num_class, learning_rate, layers, batch_size, epochs)
    plt.title(title);
    
    if save:
        filename = filename_gen('',path)
        plt.savefig(filename);

def save_f1_plot(f1, save=True, path="plots/"):
    ''' Draws and saves a plot which displays changes of F1-score
        during training for the validation set.
        F1 calculated and drawn in three types of averaging. '''
    plt.close()
    fig = plt.figure()
    x = [i*200 for i in range(len(f1[0]))]
    ax = fig.add_subplot(111)
    
    ax.plot(x,f1[0], label='F1 micro')
    ax.plot(x,f1[1], label='F1 macro')
    ax.plot(x,f1[2], label='F1 ważone')
    ax.legend(loc=7)
    ax.set_ylabel('F1')
    ax.set_xlabel('Krok')
    plt.ylim([0,1])
    vals = ax.get_yticks()
    ax.set_yticklabels(['{:3.0f}%'.format(x*100) for x in vals])
    
    title = "Cele terapeutyczne: {:2}, Szybkość uczenia: {}\nWarstwy: {}, Rozmiar batcha: {}, Liczba epok: {}".format(
        num_class, learning_rate, layers, batch_size, epochs)
    plt.title(title);
    
    if save:
        filename = filename_gen('f1',path)
        plt.savefig(filename);

def save_tf_fractions_plot(tff, save=True, path='plots/'):
    ''' Draws and saves a histogram of true/false positive/negative fractions:
        True Positive rate, False Positive Rate, False Negative Rate and
        False Discovery Rate.
        True Negative Rate not included due to having values indistinguishably
        close to 100%; however, TNR = 100% - FPR, and the FPR histogram is more
        visually clear. '''
    plt.close()
    fig = plt.figure()
    subplots = [fig.add_subplot(221), fig.add_subplot(222),
                fig.add_subplot(223), fig.add_subplot(224)]
    titles = ['Odsetek prawdziwie pozytywnych', 'Odsetek fałszywie pozytywnych',
              'Odsetek fałszywie negatywnych', 'Odsetek fałszywych odkryć']
    rate_values = [tff[0]/(tff[0]+tff[1]), tff[3]/(tff[2]+tff[3]),
                   tff[1]/(tff[0]+tff[1]), tff[3]/(tff[0]+tff[3])]
    xlabel = 'Cele terapeutyczne'
    xrange = range(1,len(tff[0])+1)
    
    for i, ax in enumerate(subplots):
        ax.bar(xrange, rate_values[i])
        ax.set_title(titles[i])
        ax.set_xlabel(xlabel)
        ax.set_xticks([])
        ax.set_xlim([0,num_class+2])
        vals = ax.get_yticks()
        if i == 1: #false positive rate is very small
            ax.set_yticklabels(['{:3.2f}%'.format(x*100) for x in vals])
        else:
            ax.set_yticklabels(['{:3.0f}%'.format(x*100) for x in vals])
    
    plt.tight_layout()
    title = "Cele terapeutyczne: {:2}, Szybkość uczenia: {}\nWarstwy: {}, Rozmiar batcha: {}, Liczba epok: {}".format(
        num_class, learning_rate, layers, batch_size, epochs)
    fig.suptitle(title)
    fig.subplots_adjust(top=0.85)
    
    if save:
        filename = filename_gen('tff',path)
        plt.savefig(filename)
        
import pickle

def load_var(name):
    with open(name,'rb') as f:
        return pickle.load(f)

def save_var(name,var):
    with open(name,'wb') as f:
        pickle.dump(var,f)

# To create new examples:

---

# Checking random seed

## Loading variables saved in files

In [2]:
dat1 = load_var('random_seed/dat1.txt')
dat2 = load_var('random_seed/dat2.txt')

tff1 = dat1['tff']
tff2 = dat2['tff']

## The rest

In [3]:
def tp(tff):
    return (tff[0]/(tff[0]+tff[1]))*100

tp_diff = abs(tp(tff1)-tp(tff2))
where = np.where(tp_diff>=5) #=[34,50,70,72]
print(where,tp_diff[where])

(array([34, 50, 70, 72]),) [  6.48854962   5.11363636   7.14285714  10.        ]


In [4]:
vpr1 = np.transpose(dat1['vpr'])[where]
vpr2 = np.transpose(dat2['vpr'])[where]

In [6]:
for i in range(4):
    w1 = np.where((vpr1[i]>.4)&(vpr1[i]<.6))
    w2 = np.where((vpr2[i]>.4)&(vpr2[i]<.6))
    print(vpr1[i][w1],'\n',vpr2[i][w1],'\n\n',vpr1[i][w2],'\n',vpr2[i][w2],'\n---')

[ 0.45830932  0.46535069  0.55543202  0.56893355  0.44075245  0.52805096
  0.54730171  0.57909626  0.40319604  0.45612869  0.43193114  0.50551683
  0.44401535  0.45406419  0.4334023   0.56521147  0.50617313  0.45471871] 
 [ 0.01807402  0.02670146  0.89881915  0.20726749  0.03037917  0.3725228
  0.0556124   0.00991256  0.13183682  0.14797866  0.00358292  0.02836937
  0.53550476  0.25896418  0.10917896  0.01448439  0.01880406  0.46757531] 

 [ 0.83434689  0.8886258   0.79487962  0.78034812  0.75695109  0.17841274
  0.93325675  0.86481446  0.44401535  0.75695109  0.99957603  0.45471871] 
 [ 0.47251084  0.59315091  0.40713882  0.44415307  0.58417165  0.46368834
  0.43259549  0.44519094  0.53550476  0.58417165  0.56925076  0.46757531] 
---
[ 0.42132023  0.42705861  0.45029482  0.59375715  0.5582127   0.56526804
  0.59111178  0.46548209  0.5431118   0.5636999   0.45649257  0.52217704
  0.46018896  0.46820307  0.568326  ] 
 [ 0.24361387  0.05599518  0.04614478  0.03191292  0.70169497  0.34303

## Classes 70 and 72 (numbered from 0) don't have predictions within the interval (0.4, 0.6)!